# Drill Parameterer classes prototyping

Prototyping drill related classes to be used to understand the interaction drill/rock. The class DrillComponent defined below was being used to compute the Reflection coefficient and delay for specific drill setups.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
from matplotlib import pyplot as plt

os.chdir('/home/bruno/datacloud')

plt.style.use('seaborn-dark')

plt.rcParams['axes.grid'] = True
plt.rcParams['figure.figsize'] = (7.5, 5)

In [ ]:
from theory.core import Pipe, Rock, TheoreticalWavelet

In [ ]:
pipe = Pipe()

In [ ]:
import pint
import numpy as np

In [ ]:
from plotly import tools

In [ ]:
DEFAULT_INPUT_UNITS = {
    "length": 'm',
    "outer_diameter": 'in',
    "inner_diameter": 'in',
    "weight": 'kg',
    "alpha": 'm/s',
    "beta": 'm/s',
    "rho": 'kg/m^3',
}

DEFAULT_OUTPUT_UNITS = {
    "length": 'm',
    "outer_diameter": 'm',
    "inner_diameter": 'm',
    "weight": 'kg',
    "alpha": 'm/s',
    "beta": 'm/s',
    "rho": 'kg/m^3',
}

ureg = pint.UnitRegistry()

class DrillComponent(object):
    '''
    A base class to be used as a component of the drill.
    '''
    def __init__(self, 
                 name=None,
                 length=None, 
                 outer_diameter=None,
                 inner_diameter=None, 
                 weight=None, 
                 rho=None,
                 alpha=None,
                 beta=None,
                 product_id=None,
                 catalogue_no=None,
                 input_units=DEFAULT_INPUT_UNITS,
                 output_units=DEFAULT_OUTPUT_UNITS,
                ):
        
        self.name = name
        self.component = component # axial or tangential
        
        # These are all physical or geometrical properties of the component wraped by pint to make it work with
        # multiple units.
        self._length = length * ureg(input_units['length'], force_ndarray=True).to('m') if length else None
        self._outer_diameter = outer_diameter * ureg(input_units['outer_diameter'], force_ndarray=True).to('m') if outer_diameter else None
        self._inner_diameter = inner_diameter * ureg(input_units['inner_diameter'], force_ndarray=True).to('m') if inner_diameter else None
        self._weight = weight * ureg(input_units['weight'], force_ndarray=True).to('kg') if weight else None
        self._rho = rho * ureg(input_units['rho'], force_ndarray=True).to('kg/m^3') if rho else None
        self._alpha = alpha * ureg(input_units['alpha'], force_ndarray=True).to('m/s') if alpha else None
        self._beta = beta * ureg(input_units['beta'], force_ndarray=True).to('m/s') if beta else None
        
        # Dictionaries holding units
        self.input_units = input_units
        self.output_units = output_units
        
        # Product information
        self.product_id = product_id
        self.catalogue_no = catalogue_no
        
        self.A1 = self.effective_area
        self.Z1 = self.impedance
        
    def set_attribute_unit(self, attribute, unit):
        '''
        Set new unit for the output dict.
        '''
        self.output_units[attribute] = unit
    
    @property
    def length(self):
        '''
        Length of the component.
        '''
        if self._length:
            return self._length.to(self.output_units['length'])
        else:
            return np.nan
        
    @property
    def outer_diameter(self):
        '''
        Outer diameter of the component.
        '''
        if self._outer_diameter:
            return self._outer_diameter.to(self.output_units['outer_diameter'])
        else:
            return np.nan
        
    @property
    def od(self):
        '''
        Outer diameter of the component.
        '''
        return self.outer_diameter
        
    @property
    def inner_diameter(self):
        '''
        Inner diameter of the component.
        '''
        if self._inner_diameter:
            return self._inner_diameter.to(self.output_units['inner_diameter'])
        else:
            if ~np.isnan(self.outer_diameter) & ~np.isnan(self.length) & ~np.isnan(self.weight) & ~np.isnan(self.rho):
                return np.sqrt(((self.od**2)-((4*self.weight)/((self.density*np.pi*self.length))))).to(self.output_units['inner_diameter'])
            pass
    
    def effective_area(self, component='axial'):                                                                  
        """                                                                        
        Cross section area. (Old A1 - Effective drill stem area for axial.)
        """      
        if component == 'axial':
            return np.pi * (((self.outer_diameter / 2) ** 2) - ((self.inner_diameter / 2) ** 2))
        if component == 'tangential':
            return np.pi * (((self.outer_diameter / 2) ** 2) + ((self.inner_diameter / 2) ** 2))
        
    @property
    def weight(self):
        '''
        Weight of the component.
        '''
        if self._weight:
            return self._weight.to(self.output_units['weight'])
        else:
            return np.nan
        
    @property
    def alpha(self):
        '''
        Compressional velocity of the component.
        '''
        if self._alpha:
            return self._alpha.to(self.output_units['alpha'])
        else:
            return np.nan
    
    @property
    def beta(self):
        '''
        Shear velocity of the component.
        '''
        if self._beta:
            return self._beta.to(self.output_units['beta'])
        else:
            return np.nan
    
    @property
    def rho(self):
        '''
        Density of the component.
        '''
        if self._rho:
            return self._rho.to(self.output_units['rho'])
        else:
            return np.nan
        
    @property
    def density(self):
        '''
        Density of the component.
        '''
        return self.rho
    
    def velocity(self, component='axial'):
        if component == 'axial':
            return self.alpha
        else:
            return self.beta
    
                                                                                   
    @property                                                                      
    def Ab(self):                                                                  
        """                                                                        
        Area of the bit contacting rock.                                           
        """                                                                        
        return np.pi * (self.Rb ** 2)
                                                                                   
    def impedance(self, component='axial'):                                                                  
        """                                                                        
        Steel impedance. (Old Z1)                                                        
        """                                                                        
        if component == 'axial':                                              
            return self.Ab * self.rho * self.alpha * (3/4)                         
        if component == 'tangential':                                         
            return self.Ab * self.rho * self.beta 

        
    def delay(self, component='axial'):
        '''
        '''
        if component == 'axial':                                              
            return 2 * self.length / self.alpha
        if component == 'tangential':                                              
            return 2 * self.length / self.beta
    
    def __repr__(self):
        return '<Drill component: {} - OD: {}, ID: {}, Length: {}>'.format(
            self.name, 
            str((np.round(self.od or np.nan, 2))),
            str((np.round(self.inner_diameter or np.nan, 2))),
            str((np.round(self.length or np.nan, 2))),
        )

In [ ]:
import pint

In [ ]:
"""
So for LCO pipe ID 7.75" and sub ID 3.5".  I think that is a good starting point for 10.75" OD steel/subs.
"""

In [ ]:
for component in ['axial', 'tangential']:
    bitsub = DrillComponent('bitsub',  length=64,  outer_diameter=10.75,  inner_diameter=3.5,  weight=930.6,
                            rho=7850, alpha=4875, beta=2368,
                    input_units={
                          'length': 'in', 
                          'outer_diameter': 'in', 
                          'inner_diameter': 'in', 
                          'weight': 'lbs', 
                          'alpha': 'm/s', 
                          'beta': 'm/s', 
                          'rho': 'kg/m^3'
                      })

    pipe = DrillComponent('pipe',  length=65,  outer_diameter=10.75,  inner_diameter=7.75,  weight=1918.4,
                            rho=7850, alpha=4875, beta=2368,
                    input_units={
                          'length': 'ft', 
                          'outer_diameter': 'in', 
                          'inner_diameter': 'in', 
                          'weight': 'lbs', 
                          'alpha': 'm/s', 
                          'beta': 'm/s', 
                          'rho': 'kg/m^3'
                      })

    RC = (np.asarray(bitsub.effective_area(component)) - np.asarray(pipe.effective_area(component))) / (np.asarray(bitsub.effective_area(component)) + np.asarray(pipe.effective_area(component)))
    print('RC for {}: {}'.format(component, RC))
    print('delay for bitsub {}: {}'.format(component, bitsub.delay(component).to('ms')))
    print('delay for pipe {}: {}'.format(component, pipe.delay(component).to('ms')))    
    print()

In [ ]:
connector = DrillComponent('pipe_connector',  length=300,  outer_diameter=10.75,  inner_diameter=5.42,  weight=1918.4,
                        rho=7850, alpha=4875, beta=2368,
                input_units={
                      'length': 'in', 
                      'outer_diameter': 'in', 
                      'inner_diameter': 'in', 
                      'weight': 'lbs', 
                      'alpha': 'm/s', 
                      'beta': 'm/s', 
                      'rho': 'kg/m^3'
                  })

In [ ]:
rock_as_component = DrillComponent('rock',  length=30,  outer_diameter=50,  inner_diameter=0.01,  weight=1918.4,
                        rho=7850, alpha=4875, beta=2368,
                input_units={
                      'length': 'in', 
                      'outer_diameter': 'in', 
                      'inner_diameter': 'in', 
                      'weight': 'lbs', 
                      'alpha': 'm/s', 
                      'beta': 'm/s', 
                      'rho': 'kg/m^3'
                  })

In [ ]:
class Drillstring(object):
    def __init__(self, drill_components, sensor_position=None):
        self.drill_components = drill_components
        self._deduplication()
        self.length = sum([component.length for component in drill_components])
        self.sensor_position = sensor_position
        
        top_position = 0* ureg('m')
        for component in self.drill_components:
            component.top_position = top_position
            component.bottom_position = top_position + component.length
            top_position += component.length
            
        self.top_positions = [dc.top_position.magnitude for dc in self]
        self.bottom_positions = [dc.bottom_position.magnitude for dc in self]
        self.drill_components_dict = dict(zip([ds.name for ds in self.drill_components], self.drill_components))

    def _deduplication(self):
        from collections import Counter
        counter_components = Counter([c.name for c in self.drill_components])
        duplicated_components = [k for k, v in counter_components.items() if v > 1]

        for duplicated_component in duplicated_components:
            counter = 1
            for i, component in enumerate(self.drill_components):
                if component.name == duplicated_component:
                    self.drill_components[i] = deepcopy(component)
                    component = self.drill_components[i]
                    component.name = component.name + ' #' + str(counter)
                    counter += 1
                    
    @property
    def component_names(self):
        return [i.name for i in self.drill_components]
    
    @property
    def first_component(self):
        return self.drill_components[0]
    
    @property
    def last_component(self):
        return self.drill_components[-1]
    
    def next_component_up(self, component_name):
        if self.component_names.index(component_name) - 1 >= 0:
            return self.component_names[self.component_names.index(component_name) - 1]
    
    def next_component_down(self, component_name):
        if self.component_names.index(component_name) + 1 < len(self.drill_components):
            return self.component_names[self.component_names.index(component_name) + 1]
    
    def get_component_on_position(self, position):
        for i, (left, right) in enumerate(zip(self.top_positions, self.bottom_positions)):
            if ((position >= left) & (position < right)):
                return self.drill_components[i]
        return None
        
    @property
    def sensor_component(self):
        return self.get_component_on_position(self.sensor_position)

                    
    def get_interfaces(self, traveling_up=True):
        interfaces = []
        drill_components = self.drill_components[::-1] if traveling_up else self.drill_components
        for i, dc in enumerate(drill_components):
            if (i + 1) < len(drill_components):
                interfaces.append((dc.name, drill_components[i + 1].name))
        return interfaces
            
    def get_interface_properties(self, traveling_up=True, component='axial'):

        interfaces = self.get_interfaces(traveling_up=traveling_up)

        interface_properties = {}
        for interface in interfaces:
            A = self[interface[0]]
            B = self[interface[1]]

            RC = (np.asarray(A.effective_area(component)) - np.asarray(B.effective_area(component))) / (np.asarray(A.effective_area(component)) + np.asarray(B.effective_area(component)))
            TC = 1 - RC

            previous_position = A.bottom_position.magnitude if traveling_up else A.top_position.magnitude
            interface_position = A.top_position.magnitude if traveling_up else A.bottom_position.magnitude
            forward_position = B.top_position.magnitude if traveling_up else B.bottom_position.magnitude

            interface_properties[interface] = {
                'RC': RC,
                'TC': TC,
                'previous_position': previous_position,
                'forward_position': forward_position,
                'interface_position': interface_position,
                'previous_component': self.get_component_on_position(previous_position),
                'forward_component': self.get_component_on_position(forward_position),
            }
        return interface_properties
    
    
    def plot(self):
        palette = sns.color_palette("pastel", len(self.drill_components))
        fig, ax = plt.subplots(1,1, figsize=(15,5))
        for dc, color in zip(self.drill_components, palette):
            width=dc.length.magnitude
            xmin = dc.top_position.magnitude
            xmax = xmin + width
            patch = plt.Rectangle((xmin, (-dc.outer_diameter/2).magnitude), 
                                  width=dc.length.magnitude, 
                                  height=dc.outer_diameter.magnitude, color=color, label=dc.name)
            ax.add_patch(patch)

            ax.hlines([dc.inner_diameter.magnitude/2, -dc.inner_diameter.magnitude/2], xmin, xmax, linestyles='dashed')

        ax.set_xlim(-1, ds.length.magnitude+1)
        ax.set_ylim(-max([c.outer_diameter for c in self.drill_components]).magnitude*3, max([c.outer_diameter for c in self.drill_components]).magnitude*3)
        ax.set_xlabel('Length ({})'.format(dc.length.units))
        ax.set_ylabel('XSection ({})'.format(dc.od.units))
        ax.legend()
        return fig, ax
        
    def __iter__(self):
        for ds in self.drill_components:
            yield ds
            
    def __getitem__(self, name):
        if name in self.drill_components_dict:
            return self.drill_components_dict[name]
        raise KeyError('Component "{}" not found.'.format(name))
        
    def __repr__(self):
        return '<Drill string - components: "{}">'.format('", "'.join([ds.name for ds in self.drill_components]))

In [ ]:
from copy import deepcopy

In [ ]:
ds = Drillstring([pipe, bitsub, rock_as_component], sensor_position=0)

In [ ]:
fig, ax = ds.plot()
ax.set_ylim(-1, 1)

In [ ]:
class WaveTrajectory(object):
    def __init__(self, 
                 wave_type='primary',
                 initial_time=0, 
                 start_position=0, 
                 end_position=10, 
                 sensor_position=8,
                 velocity=1,
                 order=0,
                ):
        self.start_position = start_position
        self.initial_time = initial_time
        self.end_position = end_position
        self.velocity = velocity
        self.wave_type = wave_type
        self.sensor_position = sensor_position
        self.order = order
        
        self.traveling_up = (self.end_position < self.start_position)
        self.traveling_down = not(self.traveling_up)
        
        self.displacement = abs(self.end_position - self.start_position)
        
    def current_displacement(self, time):
        return self.velocity * time
    
    def current_position(self, time):
        displacement = self.current_displacement(time)
        return self.start_position - displacement if self.traveling_up else self.start_position + displacement
    
    @property
    def end_time(self):
        return (self.displacement / self.velocity) + self.initial_time
    
    @property
    def displacement_to_sensor(self):
        if self.hit_sensor:
            return abs(self.sensor_position - self.start_position)
        else:
            return None        
    
    @property
    def displacement_after_sensor(self):
        if self.hit_sensor:
            return abs(self.sensor_position - self.end_position)
        else:
            return None        
    
    @property
    def hit_sensor(self):
        if self.traveling_up:
            if self.end_position <= self.sensor_position:
                return True
            else:
                return False
        if self.traveling_down:
            if self.end_position >= self.sensor_position:
                return True
            else:
                return False
        
    @property
    def time_it_hits_sensor(self):
        if self.hit_sensor:
            return self.velocity * self.displacement_to_sensor + self.initial_time
        else:
            return None
        
    def __repr__(self):
        return '<Wave Trajectory "{}" traveling from {} to {} at {} - order: {}>'.format(
            self.wave_type,
            str(self.start_position), str(self.end_position), 
            str(self.velocity), self.order)

In [ ]:
class TraceSimulation(object):
    def __init__(self, drillstring):
        self.drillstring = drillstring

In [ ]:
traveling_up = True
source = ds.last_component.bottom_position.magnitude

traveling_up_properties = ds.get_interface_properties(traveling_up=traveling_up)
traveling_down_properties = ds.get_interface_properties(traveling_up=not(traveling_up))

sensor_position = 0

current_component = ds.get_component_on_position(source-(1e-9) if traveling_up else source+(1e-9))

In [ ]:
def decompose_wave(source, traveling_up, drillstring=ds,
                      traveling_up_properties=traveling_up_properties,
                      traveling_down_properties=traveling_down_properties,
                      sensor_position=0, initial_time=0, order=0, source_index=None, from_component='rock'):
    current_component = drillstring.get_component_on_position(source-(1e-9) if traveling_up else source+(1e-9))
    if traveling_up:
        current_component = drillstring.get_component_on_position(source-(1e-9))
        if current_component:
            next_component_up = drillstring.next_component_up(current_component.name)
            if next_component_up:
                interface_it_hits = (current_component.name, next_component_up)
                properties = traveling_up_properties[(current_component.name, next_component_up)]
            else:
                return None
        else:
            return None
    else:
        current_component = drillstring.get_component_on_position(source+(1e-9))
        if current_component:
            next_component_down = drillstring.next_component_down(current_component.name)
            if next_component_down:
                interface_it_hits = (current_component.name, next_component_down)
                properties = traveling_down_properties[(current_component.name, next_component_down)]
            else:
                return None
        else:
            return None
            
    wt = WaveTrajectory(initial_time=initial_time, 
                        start_position=source, 
                        end_position=properties['interface_position'],
                        velocity=current_component.velocity().magnitude,
                        sensor_position=sensor_position,
                        order=order)
    
    while order<3:
        
    return pd.DataFrame([[source_index, source, wt.end_position, traveling_up, order, initial_time, wt.end_time, from_component, interface_it_hits, wt.hit_sensor]], 
                        columns=['source_index', 'start_position', 'end_position', 'traveling_up', 'order', 'initial_time', 'end_time','source_interface', 'interface_it_hits', 'hit_sensor'])

In [ ]:
decompose_wave(21.437599999999996, traveling_up=True, from_component=('rock', 'bitsub'))

In [ ]:
decompose_wave(19.812, traveling_up=False, from_component=('bitsub', 'pipe'), source_index=0, initial_time=0.000333, order=1)

In [ ]:
df = pd.DataFrame()

In [ ]:
for k,v in properties.items():
    if k[0] == current_component.name:


In [ ]:
wt = WaveTrajectory(initial_time=0, 
                    start_position=source, 
                    end_position=v['interface_position'],
                    velocity=current_component.velocity().magnitude,
                    sensor_position=0,
                    order=0,
                   )

new_source = v['interface_position']
forward_component = ds.get_component_on_position(new_source-(1e-9) if traveling_up else new_source+(1e-9))

rwt = WaveTrajectory(initial_time=wt.end_time,
                     start_position=v['interface_position'],
                     end_position=source,
                     velocity=current_component.velocity().magnitude,
                     sensor_position=0,
                     order=wt.order+1,
                    )

twt = WaveTrajectory(initial_time=wt.end_time,
                     start_position=v['interface_position'],
                     end_position=v['forward_position'],
                     velocity=forward_component.velocity().magnitude,
                     sensor_position=0,
                     order=wt.order+1,
                    )

In [ ]:
source

In [ ]:
rwt

In [ ]:
properties

In [ ]:
v

In [ ]:
k[0] == current_component.name

In [ ]:
current_component.name

In [ ]:
v

In [ ]:
WaveTrajectory(start_position=source, end_position=)

In [ ]:
ds.get_interface_properties()